In [25]:
import numpy  as np
from random import uniform
from copy import copy
import math

In [26]:
XMAX = 105
XMIN = -XMAX
YMAX = 195
YMIN = -90

XMID = (XMAX + XMIN) / 2.0
YMID = (YMAX + YMIN) / 2.0

START_X = 0
START_Y = 200

In [27]:
XMID

0.0

In [28]:
def clamp(value, min_value, max_value):
    return min(max(value, min_value), max_value)

In [48]:
class GCODE:
    def __init__(self, name="foobar", feedrate=1000):
        self.start_pos = np.array([START_X, START_Y])
        self.pos = copy(self.start_pos)
        self.f = open(f"{name}.gcode", "wt")
        self.feedrate = feedrate
        
        self.f.write("M17\n")
        self.f.write("M121\n")
        self.f.write("; potatolangelo\n")
        
    def pen_up(self):
        pass
    
    def pen_down(self):
        pass
        
    def move_to(self, x, y, feedrate=None):
        feedrate = feedrate or self.feedrate
        
        x = clamp(x, XMIN, XMAX)
        y = clamp(y, YMIN, YMAX)
        
        self.pos[0] = x
        self.pos[1] = y
        
        self.f.write(f"G0 X{x} Y{y} F{feedrate}\n")
        
    def step(self, dx, dy, feedrate=None):
        feedrate = feedrate or self.feedrate
        self.pos[0] += dx
        self.pos[1] += dy
        
        x = self.pos[0]
        y = self.pos[1]
        x = clamp(x, XMIN, XMAX)
        y = clamp(y, YMIN, YMAX)
        self.pos[0] = x
        self.pos[1] = y
        
        self.f.write(f"G0 X{x} Y{y} F{feedrate}\n")
        
    def square_filled(self, xmin, ymin, xmax, ymax, dy=1):
        self.move_to((xmin + xmax) / 2.0, (ymin + ymax) / 2.0)
        print(f"square filled centered on {(xmin + xmax) / 2.0} {(ymin + ymax) / 2.0}")
        self.square(xmin, ymin, xmax, ymax)
        self.move_to(xmin, ymin)
        
        x = xmin
        y = ymin
        while y < ymax:
            y += dy
            self.move_to(x, y)
            
            if x == xmin:
                x = xmax
            else:
                x = xmin
                
            self.move_to(x, y)
        
        self.move_to((xmin + xmax) / 2.0, (ymin + ymax) / 2.0)
        
    def square(self, xmin, ymin, xmax, ymax):
        self.move_to(xmin, ymin)
        self.pen_down()
        self.move_to(xmax, ymin)
        self.move_to(xmax, ymax)
        self.move_to(xmin, ymax)
        self.move_to(xmin, ymin)
        self.pen_up()
        
    def move_to_mid_point(self, feedrate=None):
        self.move_to(XMID, YMID, feedrate=feedrate)
        
    def go_to_starting_position(self):
            self.f.write(f"G0 X{self.start_pos[0]} Y{self.start_pos[1]} F{self.feedrate}\n")
            
    def flush(self):
        self.f.flush()

In [49]:
gcode = GCODE("squares", feedrate=1000)
gcode.move_to_mid_point()

square_size = 10

x = XMID
y = YMID

gcode.square_filled(x - square_size, y - square_size, x + square_size, y + square_size)
for _ in range(10):
    square_size = uniform(5, 15)
    
    x = uniform(XMIN + square_size, XMAX - square_size)
    y = uniform(YMIN - square_size, YMAX + square_size)
    
    gcode.square_filled(x - square_size, y - square_size, x + square_size, y + square_size)

gcode.move_to_mid_point()
gcode.go_to_starting_position()
gcode.flush()

square filled centered on 0.0 52.5
square filled centered on -80.05027158115259 89.98258344414982
square filled centered on -14.564686141009886 45.13050284922993
square filled centered on -43.728568264401204 141.26682604980982
square filled centered on 92.97978564095975 160.08116462425335
square filled centered on 57.164890221963574 56.637064798637724
square filled centered on -69.41800776197466 47.84672878663066
square filled centered on -33.37453190947137 -80.33249651781775
square filled centered on 89.49725260190635 107.85102478765143
square filled centered on 23.151177988723532 190.66567902603975
square filled centered on -60.92450183160461 100.70043566086022


In [31]:
gcode = GCODE("foobar", feedrate=5000)

gcode.move_to(0, 50)
gcode.move_to(-50, 50)
gcode.move_to(-50, 0)
gcode.move_to(-50, -50)
gcode.move_to(0, -50)
gcode.move_to(50, -50)
gcode.move_to(50, 0)
gcode.move_to(50, 50)
gcode.move_to(0, 50)

gcode.go_to_starting_position()

gcode.flush()

In [32]:
gcode = GCODE("random", feedrate=1000)

size = 5

gcode.move_to(XMID, YMID, feedrate=5000)

for _ in range(100):
    gcode.step(
        uniform(-size, size),
        uniform(-size, size)
    )

gcode.go_to_starting_position()

gcode.flush()

In [33]:
size = 5
radius = 0
n_steps = 360 * 2
angle = 0

noise = 2

gcode = GCODE("spiral2", feedrate=1500)
gcode.move_to(XMID, YMID, feedrate=5000)


for _ in range(n_steps):
    radius += XMAX / float(n_steps)
    
    gcode.move_to(
        radius * math.sin(angle) + XMID + uniform(-noise, noise),
        radius * math.cos(angle) + YMID + uniform(-noise, noise),
    )
    
    angle += math.radians(1)

gcode.go_to_starting_position()

gcode.flush()